In [1]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path_to_email = '../data/emails.csv'

In [13]:
df_email = pd.read_csv(path_to_email)
# rename user to member as textbook use member
df_email['member'] = df_email['user']
# string format of date to date:
df_email['week'] = pd.to_datetime(df_email['week'])
df_email = df_email.drop('user',axis=1)
df_email.head()

,emailsOpened,week,member
0,3.0,2015-06-29,1.0
1,2.0,2015-07-13,1.0
2,2.0,2015-07-20,1.0
3,3.0,2015-07-27,1.0
4,1.0,2015-08-03,1.0


<h3> Any data is a time series as long as it is recorded with a sequential time stamp </h3>
<p> <span style="color: #006994;"> The email just shows emails opened by employee per week</span> <p>

In [19]:
# step 1: nulls?
df_email[df_email['emailsOpened'] == 0]
# we see there's no null eventhough a modeling of number of emails opened
# based on timestamp should include case where emails opened == 0 too.
# check just 1 employee:
df_email[df_email['member'] == 998]
# never take the week given as a given. Always check if there's missing values.
emp998 = df_email[df_email['member'] == 998]
correct_num_weeks = (emp998.week.max() - emp998.week.min()).days / 7
# divide by 7 because the calculation is the timedelta between latest date
# and the earliest date recorded 

# add 1 because if you count it by hand there's actually 26 weeks.x

adjusted_correct_num_weeks = correct_num_weeks + 1
adjusted_correct_num_weeks

26.0

In [24]:
# since there's missing week, we need to search through other members to see
# luckily multi index can do this:
complete_idx = pd.MultiIndex.from_product((set(df_email.week),
                                           set(df_email.member)))

all_email = df_email.set_index(['week', 'member'])\
    .reindex(complete_idx, fill_value = 0)\
        .reset_index()
all_email.columns = ['week', 'member', 'EmailsOpened']
all_email[all_email.member == 998].sort_values('week')

,week,member,EmailsOpened
18325,2015-02-09,998.0,0.0
37190,2015-02-16,998.0,0.0
15630,2015-02-23,998.0,0.0
43658,2015-03-02,998.0,0.0
30722,2015-03-09,998.0,0.0
...,...,...,...
36651,2018-04-30,998.0,3.0
87317,2018-05-07,998.0,3.0
4311,2018-05-14,998.0,3.0
9162,2018-05-21,998.0,3.0


In [27]:
# Notice that we have a large number of zeros at the start. These are likely before the
# member joined the organization, so they would not have been on an email list.
# get most likely date where employee joined and left
cutoff_dates = df_email.groupby('member').week.agg(['min', 'max'])
cutoff_dates = cutoff_dates.reset_index()
cutoff_dates

,member,min,max
0,1.0,2015-06-29,2018-05-28
1,3.0,2018-03-05,2018-04-23
2,5.0,2017-06-05,2018-05-28
3,6.0,2016-12-05,2018-05-28
4,9.0,2016-07-18,2018-05-28
...,...,...,...
534,991.0,2016-10-24,2016-10-24
535,992.0,2015-02-09,2015-07-06
536,993.0,2017-09-11,2018-05-28
537,995.0,2016-09-05,2018-05-28
